In [1]:
from shapely import geometry
from shapely.geometry import Point
import glob
import geopandas as gpd
import pandas as pd 
import numpy as np
from datetime import date
from math import radians, cos, sin, asin, sqrt

import datetime

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_columns', None)
#pd.set_option('float_format', '{:f}'.format)

import snowflake.connector
import time
import datetime


C:\Users\enquistm\AppData\Roaming\Python\Python38\site-packages\snowflake\connector\options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (5.0.0), please install a version that adheres to: 'pyarrow<3.1.0,>=3.0.0; extra == "pandas"'
  warn_incompatible_dep(


### Importing US Active Wildfires Data

In [2]:
#wildfires_US= pd.read_csv('https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/CY_WildlandFire_Perimeters_ToDate/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
#wildfires_US


import geopandas as gpd

geojson_str = 'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/CY_WildlandFire_Perimeters_ToDate/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'
gdf = gpd.read_file(geojson_str, driver='GeoJSON')

In [3]:
'''gdf.columns = gdf.columns.str.replace('irwin_', '')
gdf.columns = gdf.columns.str.replace('poly_', '')
gdf.head()'''

"gdf.columns = gdf.columns.str.replace('irwin_', '')\ngdf.columns = gdf.columns.str.replace('poly_', '')\ngdf.head()"

In [4]:
gdf

,OBJECTID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_Acres_AutoCalc,poly_GlobalID,poly_Source,irwin_ABCDMisc,irwin_ADSPermissionState,irwin_CalculatedAcres,irwin_ContainmentDateTime,irwin_ControlDateTime,irwin_DailyAcres,irwin_DiscoveryAcres,irwin_DispatchCenterID,irwin_EstimatedCostToDate,irwin_FFReportApprovedByTitle,irwin_FFReportApprovedByUnit,irwin_FFReportApprovedDate,irwin_FireBehaviorGeneral,irwin_FireBehaviorGeneral1,irwin_FireBehaviorGeneral2,irwin_FireBehaviorGeneral3,irwin_FireCause,irwin_FireCauseGeneral,irwin_FireCauseSpecific,irwin_FireCode,irwin_FireDepartmentID,irwin_FireDiscoveryDateTime,irwin_FireMgmtComplexity,irwin_FireOutDateTime,irwin_FSConfinePercent,irwin_FSFullSuppPercent,irwin_FSMonitorPercent,irwin_FSPointZonePercent,irwin_FSJobCode,irwin_FSOverrideCode,irwin_GACC,irwin_ICS209ReportDateTime,irwin_ICS209RForTimePeriodFrom,irwin_ICS209RForTimePeriodTo,irwin_ICS209ReportStatus,irwin_IncidentManagementOrg,irwin_IncidentName,irwin_IncidentShortDescription,irwin_IncidentTypeCategory,irwin_IncidentTypeKind,irwin_InitialLatitude,irwin_InitialLongitude,irwin_InitialResponseAcres,irwin_InitialResponseDateTime,irwin_IrwinID,irwin_IsFireCauseInvestigated,irwin_IsFireCodeRequested,irwin_IsFSAssisted,irwin_IsMultiJurisdictional,irwin_IsReimbursable,irwin_IsTrespass,irwin_IsUnifiedCommand,irwin_LocalIncidentIdentifier,irwin_PercentContained,irwin_PercentPerToBeContained,irwin_POOCity,irwin_POOCounty,irwin_POODispatchCenterID,irwin_POOFips,irwin_POOJurisdictionalAgency,irwin_POOJurisdictionalUnit,irwin_POOJurisdUnitParentUnit,irwin_POOLandownerCategory,irwin_POOLandownerKind,irwin_POOLegalDescPrinMeridian,irwin_POOLegalDescQtr,irwin_POOLegalDescQtrQtr,irwin_POOLegalDescRange,irwin_POOLegalDescSection,irwin_POOLegalDescTownship,irwin_POOPredServiceAreaID,irwin_POOProtectingAgency,irwin_POOProtectingUnit,irwin_POOState,irwin_PredominantFuelGroup,irwin_PredominantFuelModel,irwin_PrimaryFuelModel,irwin_SecondaryFuelModel,irwin_TotalIncidentPersonnel,irwin_UniqueFireIdentifier,irwin_WFDSSDecisionStatus,irwin_CreatedBySystem,irwin_ModifiedBySystem,irwin_IsDispatchComplete,irwin_OrganizationalAssessment,irwin_StratDecisionPublishDate,irwin_GlobalID,irwin_Source,irwin_ArchivedOn,irwin_ModifiedOnDateTime_dt,irwin_CreatedOnDateTime_dt,Shape__Area,Shape__Length,GlobalID,irwin_IsCpxChild,irwin_CpxName,irwin_CpxID,geometry
0,15631,Windmill Rx,Prescribed Fire,Mixed Methods,NaN,1648828211752,1648828407035,NaN,34571.140867,{5D1C6914-D77B-48E6-88AF-928C412CF362},2022 NIFS,None,FIREREPORTING,NaN,1.645569e+12,1.645569e+12,31657.0,1.0,FLFIC,NaN,None,None,NaN,None,None,None,None,Undetermined,None,None,None,None,1641334540000,None,1.645569e+12,NaN,NaN,NaN,NaN,None,None,SACC,NaN,NaN,NaN,None,None,Windmill Rx FY22,None,RX,FI,26.072450,-81.206140,40000.0,NaN,{EFC7FEDA-F036-4F3F-8EA1-0E4B36AACD18},NaN,0,0.0,0.0,0.0,0.0,NaN,000011,NaN,NaN,None,Collier,FLFIC,12021,NPS,FLBCP,None,NPS,Federal,None,None,None,None,NaN,None,SA50,NPS,FLBCP,US-FL,None,None,None,None,NaN,2022-FLBCP-000011,No Decision,wildcad,wildcad,0,None,NaN,None,IRWIN,None,1645572426083,1641385393380,0.012607,0.638987,1dd92f3e-d260-4d35-a2e9-acd9d7497ad7,0.0,None,None,"MULTIPOLYGON (((-81.21290 26.00478, -81.21324 ..."
1,15633,Roadside East Rx,Prescribed Fire,Mixed Methods,NaN,1648656789620,1649274977128,NaN,3247.582973,{791A899F-1721-4DFA-AA07-2BB445E2DB97},2022 NIFS,None,FIREREPORTING,NaN,1.643407e+12,1.643497e+12,2350.0,1.0,FLFIC,NaN,None,None,NaN,None,None,None,None,Undetermined,None,None,None,None,1642011997000,None,1.643666e+12,NaN,NaN,NaN,NaN,None,None,SACC,NaN,NaN,NaN,None,None,EVP Roadside East,None,RX,FI,25.246430,-80.796530,4500.0,NaN,{2721B8FE-509C-42F0-B865-DB6ABA0700D6},NaN,0,0.0,0.0,0.0,0.0,NaN,000044,NaN,NaN,None,Miami-Dade,FLFIC,12086,NPS,FLEVP,None,NPS,Federal,None,None,None,None,NaN,None,SA51B,NPS,FLEVP,US-FL,None,None,None,None,NaN,2022-FLEVP-000044,No Decision,wildcad,wildcad,0

In [5]:

# extracting the features needed from the US wildfire data
gdf = gdf[['irwin_IncidentName','irwin_PercentContained', 'irwin_FireBehaviorGeneral','irwin_FireMgmtComplexity','irwin_FireCause','irwin_POOState','poly_Acres_AutoCalc',
    'irwin_DailyAcres','irwin_CreatedOnDateTime_dt', 'geometry']]
gdf

,irwin_IncidentName,irwin_PercentContained,irwin_FireBehaviorGeneral,irwin_FireMgmtComplexity,irwin_FireCause,irwin_POOState,poly_Acres_AutoCalc,irwin_DailyAcres,irwin_CreatedOnDateTime_dt,geometry
0,Windmill Rx FY22,NaN,None,None,Undetermined,US-FL,34571.140867,31657.0,1641385393380,"MULTIPOLYGON (((-81.21290 26.00478, -81.21324 ..."
1,EVP Roadside East,NaN,None,None,Undetermined,US-FL,3247.582973,2350.0,1642012296657,"POLYGON ((-80.78371 25.25059, -80.78470 25.248..."
2,EVP Flamingo Camp Ground 0125,NaN,None,None,Undetermined,US-FL,25.981285,28.0,1643115833277,"POLYGON ((-80.94307 25.13596, -80.94340 25.135..."
3,Waterhole,NaN,None,None,Undetermined,US-MO,375.591353,375.0,1644249079770,"POLYGON ((-92.73166 36.79010, -92.73080 36.789..."
4,Sugar Tree West,NaN,None,None,Undetermined,US-MO,494.421572,494.0,1644249087697,"POLYGON ((-92.73013 36.78399, -92.73013 36.783..."
...,...,...,...,...,...,...,...,...,...,...
1583,Air View,NaN,None,None,Undetermined,US-NV,9.549047,9.5,1654991165103,"POLYGON ((-117.77906 40.89169, -117.77911 40.8..."
1584,SHEEP,5.0,Extreme,Type 3 Incident,Undetermined,US-CA,773.792893,0.4,1654999418767,"POLYGON ((-117.60370 34.37742, -117.60362 34.3..."
1585,Pipeline,1.0,Extreme,Type 1 Incident,Undetermined,US-AZ,4712.639296,5000.0,1655055588740,"POLYGON ((-111.63316 35.31820, -111.62860 35.3..."
1586,Door Creek,NaN,None,None,Natural,US-AK,183.913483,184.0,1655101325340,"POLYGON ((-156.18940 61.03107, -156.18810 61.0..."


In [6]:
# getting the centriods from the polygon

#gdf['geometry'].centroid

#def centriod(poly):
#    poly = poly.centriod
#    return (ploy.x, ploy.y)
#[list(x) for x in zip(*map(centriod, gdf['geometry']))] 

gdf['lon'], gdf['lat'] = gdf['geometry'].centroid.x , gdf['geometry'].centroid.y


<ipython-input-6-899ca0ffd8e4>:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'], gdf['lat'] = gdf['geometry'].centroid.x , gdf['geometry'].centroid.y
C:\Users\enquistm\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [7]:

# changing the datetime from number series into string format yyyy-mm-dd

Date=[]
for x in gdf['irwin_CreatedOnDateTime_dt']:
    dt = datetime.datetime.fromtimestamp(x / 1e3)
    dt=dt.strftime('%Y-%m-%d %H:%M:%S')
    Date.append(dt) 
    #print(dt)
#print(Date)

gdf['irwin_CreatedOnDateTime_dt'] = Date



In [8]:
gdf

,irwin_IncidentName,irwin_PercentContained,irwin_FireBehaviorGeneral,irwin_FireMgmtComplexity,irwin_FireCause,irwin_POOState,poly_Acres_AutoCalc,irwin_DailyAcres,irwin_CreatedOnDateTime_dt,geometry,lon,lat
0,Windmill Rx FY22,NaN,None,None,Undetermined,US-FL,34571.140867,31657.0,2022-01-05 07:23:13,"MULTIPOLYGON (((-81.21290 26.00478, -81.21324 ...",-81.191814,25.940551
1,EVP Roadside East,NaN,None,None,Undetermined,US-FL,3247.582973,2350.0,2022-01-12 13:31:36,"POLYGON ((-80.78371 25.25059, -80.78470 25.248...",-80.796542,25.240981
2,EVP Flamingo Camp Ground 0125,NaN,None,None,Undetermined,US-FL,25.981285,28.0,2022-01-25 08:03:53,"POLYGON ((-80.94307 25.13596, -80.94340 25.135...",-80.945578,25.136301
3,Waterhole,NaN,None,None,Undetermined,US-MO,375.591353,375.0,2022-02-07 10:51:19,"POLYGON ((-92.73166 36.79010, -92.73080 36.789...",-92.740728,36.784081
4,Sugar Tree West,NaN,None,None,Undetermined,US-MO,494.421572,494.0,2022-02-07 10:51:27,"POLYGON ((-92.73013 36.78399, -92.73013 36.783...",-92.735842,36.777491
...,...,...,...,...,...,...,...,...,...,...,...,...
1583,Air View,NaN,None,None,Undetermined,US-NV,9.549047,9.5,2022-06-11 19:46:05,"POLYGON ((-117.77906 40.89169, -117.77911 40.8...",-117.777806,40.893347
1584,SHEEP,5.0,Extreme,Type 3 Incident,Undetermined,US-CA,773.792893,0.4,2022-06-11 22:03:38,"POLYGON ((-117.60370 34.37742, -117.60362 34.3...",-117.610327,34.366805
1585,Pipeline,1.0,Extreme,Type 1 Incident,Undetermined,US-AZ,4712.639296,5000.0,2022-06-12 13:39:48,"POLYGON ((-111.63316 35.31820, -111.62860 35.3...",-111.627554,35.305676
1586,Door Creek,NaN,None,None,Natural,US-AK,183.913483,184.0,2022-06-13 02:22:05,"POLYGON ((-156.18940 61.03107, -156.18810 61.0...",-156.200351,61.027851


In [9]:
# remove 'US-'' out from a the string 

print(gdf['irwin_POOState'].unique())

gdf['irwin_POOState'] = gdf['irwin_POOState'].str.replace('US-','')

print(gdf['irwin_POOState'].unique())

['US-FL' 'US-MO' 'US-TN' 'US-UT' 'US-NM' 'US-SC' 'US-NC' 'US-MA' 'US-OK'
 'US-WI' 'US-WY' 'US-ME' 'US-LA' 'US-TX' 'US-NE' 'US-AR' 'US-CA' 'US-NV'
 'US-SD' 'US-MS' 'US-MT' 'US-AZ' 'US-AL' 'US-GA' 'US-ND' 'US-OR' 'US-KS'
 'US-VA' 'US-KY' 'US-IL' 'US-MD' 'US-WA' 'US-OH' 'US-IN' 'US-WV' 'US-IA'
 'US-CO' 'US-MN' 'US-ID' 'US-AK' 'US-MI' 'US-NJ' 'US-PA' 'US-VT']
['FL' 'MO' 'TN' 'UT' 'NM' 'SC' 'NC' 'MA' 'OK' 'WI' 'WY' 'ME' 'LA' 'TX'
 'NE' 'AR' 'CA' 'NV' 'SD' 'MS' 'MT' 'AZ' 'AL' 'GA' 'ND' 'OR' 'KS' 'VA'
 'KY' 'IL' 'MD' 'WA' 'OH' 'IN' 'WV' 'IA' 'CO' 'MN' 'ID' 'AK' 'MI' 'NJ'
 'PA' 'VT']


C:\Users\enquistm\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [10]:
 # acres to hectares
    
gdf['poly_Acres_AutoCalc']  = gdf['poly_Acres_AutoCalc']  / 2.47715769 

In [11]:
gdf['irwin_FireMgmtComplexity'].unique()

array([None, 'Type 5 Incident', 'Type 4 Incident', 'Type 3 Incident',
       'Type 2 Incident', 'Type 1 Incident'], dtype=object)

In [12]:
gdf['poly_Acres_AutoCalc'].describe()

count      1588.000000
mean        631.065057
std        5209.486723
min           0.000009
25%           0.399932
50%           4.414994
75%          43.005993
max      129379.505952
Name: poly_Acres_AutoCalc, dtype: float64

#### Creating the stage_of_control feature column that is similar to Canadian wildfire 

In [13]:
gdf

,irwin_IncidentName,irwin_PercentContained,irwin_FireBehaviorGeneral,irwin_FireMgmtComplexity,irwin_FireCause,irwin_POOState,poly_Acres_AutoCalc,irwin_DailyAcres,irwin_CreatedOnDateTime_dt,geometry,lon,lat
0,Windmill Rx FY22,NaN,None,None,Undetermined,FL,13955.970993,31657.0,2022-01-05 07:23:13,"MULTIPOLYGON (((-81.21290 26.00478, -81.21324 ...",-81.191814,25.940551
1,EVP Roadside East,NaN,None,None,Undetermined,FL,1311.011804,2350.0,2022-01-12 13:31:36,"POLYGON ((-80.78371 25.25059, -80.78470 25.248...",-80.796542,25.240981
2,EVP Flamingo Camp Ground 0125,NaN,None,None,Undetermined,FL,10.488345,28.0,2022-01-25 08:03:53,"POLYGON ((-80.94307 25.13596, -80.94340 25.135...",-80.945578,25.136301
3,Waterhole,NaN,None,None,Undetermined,MO,151.621899,375.0,2022-02-07 10:51:19,"POLYGON ((-92.73166 36.79010, -92.73080 36.789...",-92.740728,36.784081
4,Sugar Tree West,NaN,None,None,Undetermined,MO,199.592288,494.0,2022-02-07 10:51:27,"POLYGON ((-92.73013 36.78399, -92.73013 36.783...",-92.735842,36.777491
...,...,...,...,...,...,...,...,...,...,...,...,...
1583,Air View,NaN,None,None,Undetermined,NV,3.854840,9.5,2022-06-11 19:46:05,"POLYGON ((-117.77906 40.89169, -117.77911 40.8...",-117.777806,40.893347
1584,SHEEP,5.0,Extreme,Type 3 Incident,Undetermined,CA,312.371270,0.4,2022-06-11 22:03:38,"POLYGON ((-117.60370 34.37742, -117.60362 34.3...",-117.610327,34.366805
1585,Pipeline,1.0,Extreme,Type 1 Incident,Undetermined,AZ,1902.438151,5000.0,2022-06-12 13:39:48,"POLYGON ((-111.63316 35.31820, -111.62860 35.3...",-111.627554,35.305676
1586,Door Creek,NaN,None,None,Natural,AK,74.243753,184.0,2022-06-13 02:22:05,"POLYGON ((-156.18940 61.03107, -156.18810 61.0...",-156.200351,61.027851


In [14]:
gdf['irwin_FireCause'].unique()

array(['Undetermined', 'Human', None, 'Natural'], dtype=object)

In [15]:

# creating dummy variables for null values
gdf['irwin_PercentContained_null'] = np.where(gdf['irwin_PercentContained'].isnull(),1,0)
gdf['irwin_FireBehaviorGeneral_null'] = np.where(gdf['irwin_FireBehaviorGeneral'].isnull(),1,0)
gdf['irwin_FireMgmtComplexity_null'] = np.where(gdf['irwin_FireMgmtComplexity'].isnull(),1,0)
gdf['irwin_FireCause_null'] = np.where(gdf['irwin_FireCause'].isnull(),1,0)



# setting up the logic   
gdf['stage_of_control'] = 'N/A'

gdf.loc[gdf['irwin_FireBehaviorGeneral_null'].eq(1) & (gdf['poly_Acres_AutoCalc'] >= gdf['poly_Acres_AutoCalc'].quantile(.75)) , 'stage_of_control'] = 'OC'
gdf.loc[gdf['irwin_FireBehaviorGeneral_null'].eq(1) & (gdf['poly_Acres_AutoCalc'] <= gdf['poly_Acres_AutoCalc'].quantile(.75)) , 'stage_of_control'] = 'UC'


gdf.loc[gdf['irwin_FireBehaviorGeneral_null'].eq(0) & (gdf['poly_Acres_AutoCalc'] >= gdf['poly_Acres_AutoCalc'].quantile(.75)) , 'stage_of_control'] = 'OC'
gdf.loc[gdf['irwin_FireBehaviorGeneral_null'].eq(0) & (gdf['poly_Acres_AutoCalc'] <= gdf['poly_Acres_AutoCalc'].quantile(.75)) , 'stage_of_control'] = 'UC'





gdf.loc[ (gdf['irwin_PercentContained']>=85), 'stage_of_control'] = 'UC'
gdf.loc[ (gdf['irwin_PercentContained']<85), 'stage_of_control'] = 'OC'


gdf.loc[gdf['irwin_FireMgmtComplexity'].isin(['Type 5 Incident', 'Type 4 Incident']) , 'stage_of_control'] = 'UC'
gdf.loc[gdf['irwin_FireMgmtComplexity'].isin(['Type 1 Incident', 'Type 2 Incident', 'Type 3 Incident']) , 'stage_of_control'] = 'OC'



gdf.loc[gdf['irwin_FireBehaviorGeneral_null'].eq(0)    , 'stage_of_control'] = 'UC'





gdf.loc[gdf['irwin_FireMgmtComplexity'].isin(['Type 5 Incident', 'Type 4 Incident']) , 'stage_of_control'] = 'UC'
gdf.loc[gdf['irwin_FireMgmtComplexity'].isin(['Type 1 Incident', 'Type 2 Incident', 'Type 3 Incident']) , 'stage_of_control'] = 'OC'

gdf.loc[ (gdf['irwin_PercentContained']>=85), 'stage_of_control'] = 'UC'
gdf.loc[ (gdf['irwin_PercentContained']<85), 'stage_of_control'] = 'OC'


gdf['stage_of_control'].value_counts()


C:\Users\enquistm\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\enquistm\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\enquistm\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

UC    1449
OC     139
Name: stage_of_control, dtype: int64

In [16]:
# Percentage contained column

gdf['stage_of_control'] = np.where( gdf['irwin_PercentContained_null'].eq(0),  gdf['irwin_PercentContained'].astype(str) +" % "+ gdf['stage_of_control'] , gdf['stage_of_control'])

#gdf['stage_of_control'].unique()

C:\Users\enquistm\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)



pd.options.display.max_rows = 1000

print('\n')
print('Length for gdf Table:',len(gdf))
print('\n')

print('\n')
print("gdf Unique Values:",'\n',gdf[gdf.columns].nunique())
print('\n')

print('Null Columns: ',len(gdf.columns[gdf.isnull().sum() >1] ))
print('\n')
print('Percentage of Zero Columns: ','\n',(gdf == 0).astype(int).sum(axis=0)/len(gdf)*100)
print('\n')
print("gdf Table Null Values:",'\n',gdf[gdf.columns].isnull().sum())


In [17]:
# renaming columns
gdf.rename(columns={'irwin_IncidentName':'firename', 'irwin_POOState':'agency', 'poly_Acres_AutoCalc':'hectares', 'irwin_CreatedOnDateTime_dt':'startdate'}, inplace=True)

C:\Users\enquistm\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [18]:
# retriving the main feature of the data
gdf=  gdf[['agency', 'firename', 'lat', 'lon', 'startdate', 'hectares',
       'stage_of_control']]

In [19]:
#gdf

import pandas_profiling

gdf_ =gdf.select_dtypes(exclude= 'geometry' )



#profile = pandas_profiling.ProfileReport(gdf_ , title=' Profiling Report', explorative=True)
#profile.to_file("US Active Wildfires Descriptive Statistics .html")
gdf_ =gdf_.select_dtypes(include= 'object' )

profile = pandas_profiling.ProfileReport(gdf_ , title=' Profiling Report', explorative=True)
profile.to_file("US Active Wildfires Descriptive Statistics (object) .html")


### Importing Canada Active Wildfires Data

In [20]:

wildfires= pd.read_csv('https://cwfis.cfs.nrcan.gc.ca/downloads/activefires/activefires.csv')
wildfires

,agency,firename,lat,lon,startdate,hectares,stage_of_control
0,pc,2022RM1,50.7606,-99.6775,2022-05-25 14:37:57,400.00,UC
1,nt,FS004-22,63.7368,-121.3690,2022-06-02 22:49:00,190.00,OC
2,yt,2022BC002,62.2201,-138.9020,2022-06-06 01:22:00,68.70,OC
3,nt,FS003-22,62.1872,-121.9400,2022-06-02 22:58:00,1211.00,OC
4,yt,2022WL001,60.0275,-128.7450,2022-05-23 03:49:00,2.00,UC
...,...,...,...,...,...,...,...
74,mb,WE004,54.7847,-101.8660,2022-06-12 05:00:00,0.20,UC
75,pc,2022WB4,59.7851,-112.3430,2022-06-12 23:44:10,1.60,BH
76,pc,2022WB3,59.8045,-112.1720,2022-06-11 21:39:52,0.70,UC
77,sk,22SR-SHARGORY,59.9028,-104.9410,2022-06-13 06:00:00,10.00,OC


In [21]:
wildfires.rename(columns=lambda x: x.strip(), inplace=True)

In [22]:
wildfires = wildfires[wildfires['lon'] <0]

In [23]:
wildfires['agency'].unique()

array(['pc', 'nt', 'yt', 'bc', 'ns', 'ab', 'ak', 'sk', 'conus', 'mb',
       'qc'], dtype=object)

In [24]:
# upper case characters
wildfires['agency']= wildfires['agency'].str.upper()

In [25]:
# fill in zeros
wildfires['hectares'] = wildfires['hectares'].fillna(0)

In [26]:
#Get today's date
today = date.today()
str(today)
print("Today's date:", today)

#Create a list of growing counts for today.
today_data_list = [str(today)]
today_data_list

Today's date: 2022-06-13


['2022-06-13']

In [27]:
wildfires['DATE'] =  today
gdf['DATE'] =  today

<ipython-input-27-456f6f5d61c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['DATE'] =  today


In [28]:
wildfires.columns

Index(['agency', 'firename', 'lat', 'lon', 'startdate', 'hectares',
       'stage_of_control', 'DATE'],
      dtype='object')

### Concating US and Canadian Wildfire tables



In [29]:
wildfires = pd.concat([wildfires,gdf], axis=0)

In [30]:
#wildfires

## Exporting flatfile into external environment

In [31]:
wildfires.to_csv("/****/activefires " + str(today) + ".csv", index=False)

In [32]:
wildfires

,agency,firename,lat,lon,startdate,hectares,stage_of_control,DATE
0,PC,2022RM1,50.760600,-99.677500,2022-05-25 14:37:57,400.000000,UC,2022-06-13
1,NT,FS004-22,63.736800,-121.369000,2022-06-02 22:49:00,190.000000,OC,2022-06-13
2,YT,2022BC002,62.220100,-138.902000,2022-06-06 01:22:00,68.700000,OC,2022-06-13
3,NT,FS003-22,62.187200,-121.940000,2022-06-02 22:58:00,1211.000000,OC,2022-06-13
4,YT,2022WL001,60.027500,-128.745000,2022-05-23 03:49:00,2.000000,UC,2022-06-13
...,...,...,...,...,...,...,...,...
1583,NV,Air View,40.893347,-117.777806,2022-06-11 19:46:05,3.854840,UC,2022-06-13
1584,CA,SHEEP,34.366805,-117.610327,2022-06-11 22:03:38,312.371270,5.0 % OC,2022-06-13
1585,AZ,Pipeline,35.305676,-111.627554,2022-06-12 13:39:48,1902.438151,1.0 % OC,2022-06-13
1586,AK,Door Creek,61.027851,-156.200351,2022-06-13 02:22:05,74.243753,OC,2022-06-13


In [33]:
# Rename latitidue adn longitude
wildfires.rename(columns={'lat':'LATITUDE', 'lon':'LONGITUDE' },inplace=True)

In [34]:
wildfires.columns

Index(['agency', 'firename', 'LATITUDE', 'LONGITUDE', 'startdate', 'hectares',
       'stage_of_control', 'DATE'],
      dtype='object')

In [35]:
wildfires.describe()

,LATITUDE,LONGITUDE,hectares
count,1667.000000,1667.000000,1667.000000
mean,39.557164,-99.326827,783.698024
std,9.041097,19.748381,5557.709128
min,25.136301,-176.620792,0.000009
25%,32.795683,-106.492498,0.431236
50%,36.904380,-96.754581,4.646079
75%,44.484872,-88.926486,49.981881
max,66.279757,-63.184100,129379.505952


In [36]:
wildfires = wildfires[['firename','agency', 'LATITUDE', 'LONGITUDE']]

## Building data

In [37]:

start = time.time()
# Gets the version
ctx = snowflake.connector.connect(
        user='****',
        password='****',
        account='****',
        warehouse='****',
        database='****'
        #schema= 'DIM'
    )
cs = ctx.cursor()
try:
    
    query = "USE ROLE ****_READER"





    cur = ctx.cursor().execute(query)
    query ='''SELECT B.DIM.C.CLIENT_ID, B.DIM.C.CLIENT_NAME, B.DIM.B.BUILDING_ID, B.DIM.UILDING.ADDRESS,B.DIM.B.ADDRESS2,
    B.DIM.B.CITY,
B.DIM.B.BUILDING_NAME,
B.DIM.UILDING.PROVINCE,
B.DIM.B.COUNTRY,
B.DIM.B.LATITUDE,
B.DIM.B.LONGITUDE,
B.DIM.B.POSTAL, 
B.DIM.B.AREA_SQFT,
B.DIM.B.GROSS_SQFT,
B.DIM.B.RENTABLE_SQFT,
B.DIM.B.USABLE_SQFT,
B.DIM.B.OFFICE_SPACE_AREA, 
B.DIM.B.LAND_AREA, 
B.DIM.B.GEO_FENCE_UOM, 
B.DIM.B.GEO_FENCE_RADIUS, 
B.DIM.B.BUILDING_CREATED_DATE, 
B.DIM.B.BUILDING_UPDATED_DATE, 
B.DIM.B.PHYSICALLOCATION_PRIMARY_ADDRESS, 
B.DIM.B.PHYSICALLOCATION_ADDRESS2, 
B.DIM.B.PHYSICALLOCATION_CITY, 
B.DIM.B.PHYSICALLOCATION_POSTALCODE, 
B.DIM.B.HERITAGE_CODE, 
B.DIM.B.PHYSICALLOCATION_CODE, 
B.DIM.B.HERITAGE_DESIGNATION_DATE, 
B.DIM.B.HERITAGE_DESIGNATION_DESCRIPTION, 
B.DIM.B.DATUM, 
B.DIM.B.PROPERTYUSAGE_DESC,
B.DIM.B.ACTIVEIND
  FROM  B.DIM.B INNER JOIN
                         B.DIM.C ON B.DIM.B.CLIENT_ID = B.DIM.C.CLIENT_ID 
WHERE  (DIM.C.ACTIVEIND = TRUE)                         
                         
'''
    cur = ctx.cursor().execute(query)
    Building = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

finally:
    cs.close()
ctx.close()

print(len(Building))

end = time.time()

print(str(datetime.timedelta(seconds=(end - start))))



107673
0:00:05.232515


'import winsound\nduration = 2500  # milliseconds\nfreq = 440  # Hz\nwinsound.Beep(freq, duration)'

In [38]:
print(len(Building))
Building = Building[Building['LONGITUDE'].notnull()]
print(len(Building))

107673
107673


In [39]:
#Building1 = Building.copy()
#Building = Building1.copy()

In [1]:
np.sort(Building['CLIENT_NAME'].unique())

NameError: name 'np' is not defined

In [2]:
# Retriving Building Latitdue and Longitude cleanup file
Building_ = pd.concat([pd.read_csv(name) for name in glob.glob('/****/*building_data.csv')])
print(len(Building_))
Building_.drop_duplicates('BUILDING_ID', keep='first', inplace=True)
print(len(Building_))


Building_ = Building_[Building_['LATITUDE'].notnull()]
Building_ = Building_[Building_['LONGITUDE'].notnull()]
Building_

NameError: name 'pd' is not defined

In [42]:
Building["BUILDING_ID"].nunique()

107673

print(len(Building['BUILDING_ID'].isin(Building_["BUILDING_ID"].unique()))) 

Building[Building['BUILDING_ID'].isin(Building_["BUILDING_ID"].unique() )  & (Building['COUNTRY'].isin(['Canada']))]

In [43]:

# replacing values with new lat and long data
print(len(Building[Building['LONGITUDE'].isnull()]))
Building['LATITUDE_'] = Building.BUILDING_ID.map(Building_.set_index('BUILDING_ID')['LATITUDE'].to_dict())
Building['LONGITUDE_'] = Building.BUILDING_ID.map(Building_.set_index('BUILDING_ID')['LONGITUDE'].to_dict())


#Building.loc[Building['BUILDING_ID'].isin(Building_["BUILDING_ID"].unique()) & (Building['COUNTRY'].isin(['Canada'])) , 'LATITUDE'] = Building_["LATITUDE"]
#Building.loc[Building['BUILDING_ID'].isin(Building_["BUILDING_ID"].unique())  & (Building['COUNTRY'].isin(['Canada'])), 'LONGITUDE'] = Building_["LONGITUDE"]

Building["LATITUDE"] = np.where(Building['BUILDING_ID'].isin(Building_["BUILDING_ID"].unique()),Building["LATITUDE_"],Building["LATITUDE"])

Building["LONGITUDE"] = np.where(Building['BUILDING_ID'].isin(Building_["BUILDING_ID"].unique()),Building["LONGITUDE_"],Building["LONGITUDE"])

Building.drop(columns=['LATITUDE_','LONGITUDE_'], inplace=True)

print(len(Building[Building['LONGITUDE'].isnull()]))



0
0


## Retrieving the Critical Environments data

In [44]:
CE = pd.concat([pd.read_csv(name) for name in glob.glob('/****/*CE Site Information.csv')])

# uppercase the string values
CE['Account'] = CE['Account'].str.upper()
Building['CLIENT_NAME'] = Building['CLIENT_NAME'].str.upper()



CE['Building ID'] = np.where((~CE['Building ID'].isin(Building['BUILDING_ID'])), CE['CE Group ID'], CE['Building ID'])


d = {'HONI':'HYDRO ONE', 'TD BANK':'TD CANADA'}

CE['Account'] = CE['Account'].replace(d)


d = {'US':'United States'}

CE['Country'] = CE['Country'].replace(d)


d = { 'Alabama':'AL', 'Alberta':'AB', 'Arizona':'AZ',
'British Columbia':'BC','California':'CA', 
       'Florida':'FL', 'Georgia':'GA', 'Illinois':'IL', 'Kansas':'KS', 'Maine':'ME', 'Manitoba':'MN', 'Maryland':'MD', 
     'Massachusetts':'MA',
       'Nevada':'NV', 'New Brunswick':'NB', 'New Jersey':"NJ", 'New Mexico':'NM',
        'Newfoundland':'NL', 'North Carolina':'NC',
       'Northwest Territories':'NT', 'Nova Scotia':'NS', 'Ontario':'ON',
       'Prince Edward Island':'PE', 'Quebec':'QC', 'Saskatchewan':'SK',
        'South Carolina':'SC', 'Tennessee':'TN',
       'Texas':'TX', 'Utah':'UT', 'Washington':'WA',
        'Wyoming':'WY'}

CE['ProvinceState'] = CE['ProvinceState'].replace(d)

print(CE['ProvinceState'].unique())

CE.rename(columns={'Account':'CLIENT_NAME','Building ID':'BUILDING_ID','Building':'BUILDING_NAME',   'Address':'ADDRESS',
 'Latitude':'LATITUDE',
 'Longitude':'LONGITUDE',
 'City':"CITY",
 'ProvinceState':'PROVINCE',
 'Country':'COUNTRY',

 'Building Sq. Ft.' :'RENTABLE_SQFT'}, inplace=True)

CE.columns




#Building.loc[Building['BUILDING_ID'].isin(CE["BUILDING_ID"].unique()), 'LATITUDE'] = CE["LATITUDE"]
#Building.loc[Building['BUILDING_ID'].isin(CE["BUILDING_ID"].unique()), 'LONGITUDE'] = CE["LONGITUDE"]


Building['LATITUDE_'] = Building.BUILDING_ID.map(CE.set_index('BUILDING_ID')['LATITUDE'].to_dict())
Building['LONGITUDE_'] = Building.BUILDING_ID.map(CE.set_index('BUILDING_ID')['LONGITUDE'].to_dict())




Building["LATITUDE"] = np.where(Building['BUILDING_ID'].isin(CE["BUILDING_ID"].unique()),Building["LATITUDE_"],Building["LATITUDE"])

Building["LONGITUDE"] = np.where(Building['BUILDING_ID'].isin(CE["BUILDING_ID"].unique()),Building["LONGITUDE_"],Building["LONGITUDE"])

Building.drop(columns=['LATITUDE_','LONGITUDE_'], inplace=True)


CE = CE[['CLIENT_NAME', 'BUILDING_NAME', 'BUILDING_ID', 'ADDRESS', 'LATITUDE', 'LONGITUDE', 'CITY', 'PROVINCE',
       'COUNTRY',
       'Critical Environment Type','RENTABLE_SQFT']]

# concating Critcal environmants buidlign swith Dim.Building table
Building = pd.concat([Building,CE[(~CE['BUILDING_ID'].isin(Building['BUILDING_ID'])) &  (CE['COUNTRY'].isin(['Canada','United States']))]], axis=0)

['AB' 'BC' 'NL' 'NS' 'NB' 'MN' 'QC' 'ON' 'PE' 'NT' 'SK' 'TX' 'England'
 'WY' 'MA' 'New South Wales' 'Victoria' 'UT' 'NV' 'CA' 'AZ' 'VA' 'IL' 'GA'
 'WA' 'NJ' 'Worcestershire' 'MD' 'AL' 'NM' 'TN' 'Queensland'
 'Australian Capital Territory' 'Tasmania' 'Western Australia' 'Dublin'
 'Louth' 'Beijing Shi' 'Shanghai Shi' 'Esch-sur-Alzette' 'KS' 'SC' 'ME'
 'NC' 'FL']


In [45]:
print(len(Building))
Building = Building[Building['LONGITUDE'].notnull()]
print(len(Building))

107710
107709


In [46]:
Building['PROVINCE'].unique()

array(['QC', 'ON', 'AB', 'FL', 'MB', 'ENG', 'WY', 'BC', 'NB', 'TN', 'SK',
       'AZ', 'TX', 'CA', 'NL', 'MA', 'NH', 'CT', 'NJ', 'SC', 'NY', 'VT',
       'ME', 'DC', 'MD', 'RI', 'NC', 'NM', 'NS', 'NU', 'PA', 'YT', 'VA',
       'IL', 'NT', 'PE', 'WA', 'CO', 'WI', 'DE', 'NA', 'N/A', 'SA', 'QL',
       'SH', 'VI', 'MI', 'TY', 'GP', 'OR', 'UT', 'MN', 'AR', 'LA', 'NV',
       'IN', 'WV', 'OK', 'OH', 'BAC', 'GA', 'KS', 'LI', 'M2', 'AL', 'MO',
       'NE', 'IA', 'KY', 'SD', 'A2', 'RM', 'SCT', 'WAL', 'NIR', 'MS',
       'B2', 'CD', 'LD', 'ND'], dtype=object)

In [47]:
Building['COUNTRY'].unique()

array(['Canada', 'United States', 'UK', 'AU', 'NZ', 'CN', 'KR', 'SG',
       'IN', 'JP', 'MY', 'ZA', 'MX', 'PE', 'CO', 'PA', 'LU', 'DO', 'AR',
       'CL', 'BR', 'RU', 'IL', 'FR'], dtype=object)

In [3]:
np.sort(Building['CLIENT_NAME'].unique().tolist())

NameError: name 'np' is not defined

In [49]:
Building = Building[~Building['CLIENT_NAME'].str.contains('DEMO')]

In [50]:
dataframe1 = Building.copy()

In [52]:
wildfires.columns

Index(['firename', 'agency', 'LATITUDE', 'LONGITUDE'], dtype='object')

In [53]:
# Mapping in Building and wildfires data
wildfires['BUILDING_ID_'] =  wildfires['firename']

dataframe1['BUILDING_ID_'] = dataframe1['BUILDING_ID']


dataframe1 =pd.concat([dataframe1,wildfires[['LATITUDE', 'LONGITUDE', 'BUILDING_ID_']]], axis=0)

<ipython-input-53-10d89edcef65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wildfires['BUILDING_ID_'] =  wildfires['firename']


In [54]:
# Using geopandas for lat and long coordinates for Buidling and Wildfire data


Geometry = [Point(xy) for xy in zip(dataframe1['LONGITUDE'],dataframe1['LATITUDE'], ) ]

crs= {'init':'epsg:4326'}
dataframe1 = gpd.GeoDataFrame(dataframe1, crs= crs, geometry =Geometry )
dataframe1

dataframe1 = dataframe1.to_crs('EPSG:4326')


C:\Users\enquistm\Anaconda3\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [4]:
# Using geopandas for lat and long coordinates for wildfire data
Geometry = [Point(xy) for xy in zip(wildfires['LONGITUDE'],wildfires['LATITUDE'], )]


crs= {'init':'epsg:4326'}
wildfires = gpd.GeoDataFrame(wildfires, crs= crs, geometry =Geometry )
wildfires

wildfires = wildfires.to_crs('EPSG:4326')
wildfires

NameError: name 'wildfires' is not defined

## Using Geopandas KNN for lat and long distancing inner join function

In [5]:


# Now convert the Dataframes to WebMercator
df2 = dataframe1.to_crs('EPSG:3857')
df1 = wildfires.to_crs('EPSG:3857')

# Create a buffer with a radius of 10000 meters around each point in df2
df2.geometry = df2.geometry.buffer(100000)

# Join the two Dataframes and convert back to original projection
df3 = gpd.sjoin(df1, df2, how='inner', op='intersects', lsuffix='_df1', rsuffix='_df2')
df3.to_crs('EPSG:4326') # or whatever was used originally

NameError: name 'dataframe1' is not defined

In [57]:
# check for null values in Lat and Long
df3.isnull().sum()

firename                                 0
agency                                   0
LATITUDE__df1                            0
LONGITUDE__df1                           0
BUILDING_ID___df1                        0
geometry                                 0
index__df2                               0
CLIENT_ID                            31634
CLIENT_NAME                          31541
BUILDING_ID                          31541
ADDRESS                              31541
ADDRESS2                             31634
CITY                                 31541
PROVINCE                             31541
COUNTRY                              31541
LATITUDE__df2                            0
LONGITUDE__df2                           0
POSTAL                               31637
AREA_SQFT                            31634
GROSS_SQFT                           31634
RENTABLE_SQFT                        31626
USABLE_SQFT                          31634
OFFICE_SPACE_AREA                   113420
LAND_AREA  

In [58]:
df3.columns.tolist()

['firename',
 'agency',
 'LATITUDE__df1',
 'LONGITUDE__df1',
 'BUILDING_ID___df1',
 'geometry',
 'index__df2',
 'CLIENT_ID',
 'CLIENT_NAME',
 'BUILDING_ID',
 'ADDRESS',
 'ADDRESS2',
 'CITY',
 'PROVINCE',
 'COUNTRY',
 'LATITUDE__df2',
 'LONGITUDE__df2',
 'POSTAL',
 'AREA_SQFT',
 'GROSS_SQFT',
 'RENTABLE_SQFT',
 'USABLE_SQFT',
 'OFFICE_SPACE_AREA',
 'LAND_AREA',
 'GEO_FENCE_UOM',
 'GEO_FENCE_RADIUS',
 'BUILDING_CREATED_DATE',
 'BUILDING_UPDATED_DATE',
 'PHYSICALLOCATION_PRIMARY_ADDRESS',
 'PHYSICALLOCATION_ADDRESS2',
 'PHYSICALLOCATION_CITY',
 'PHYSICALLOCATION_POSTALCODE',
 'HERITAGE_CODE',
 'PHYSICALLOCATION_CODE',
 'HERITAGE_DESIGNATION_DATE',
 'HERITAGE_DESIGNATION_DESCRIPTION',
 'DATUM',
 'PROPERTYUSAGE_DESC',
 'ACTIVEIND',
 'BUILDING_NAME',
 'Critical Environment Type',
 'BUILDING_ID___df2']

### Creating a dictance KM's away from each wilfire and building site

In [59]:


features = []
distance = []
objects = df3[['LATITUDE__df1','LONGITUDE__df1','LATITUDE__df2', 'LONGITUDE__df2']].values.tolist()

def check_point(objects):
    for i in objects:
        
        lat1_ = i[0]
        lon1_ = i[1]
        lat2_ = i[2]
        lon2_ = i[3]
        #address_ = i[4]
        #serviceprovider_ = i[5]
        #Clusters_ = i[6]

        # convert decimal degrees to radians 
        lon1, lat1, lon2, lat2 = map(radians, [lon1_, lat1_, lon2_, lat2_])

        # haversine formula 
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers. Use 3956 for miles
        dist =  c * r

        distance.append(dist)
        #address.append(address_)
        #serviceprovider.append(serviceprovider_)
        #Clusters.append(Clusters_)

        radius = 100 # in kilometer

        #print('Distance (km) : ', dist)
        if dist <= radius:
            #print('Inside the area')
            features.append(1)
        else:
            #print('Outside the area')
            features.append(0)


            
check_point(objects)




In [60]:
# Embedding the featureas and distance inside the datatable

In [61]:
df3['# of Sites (100Km Away)'] = features
df3['Distance (Km)'] = distance


In [6]:
df3

NameError: name 'df3' is not defined

In [63]:
# the total number of buildings nearby a wildfire site  

df3['# of Sites (100Km Away)'] = df3['# of Sites (100Km Away)'].astype(np.int64)


df = df3.groupby(['BUILDING_ID___df2']).sum()[['# of Sites (100Km Away)']].reset_index()

df.sort_values('# of Sites (100Km Away)', ascending=False, inplace=True)

df.rename(columns={'# of Sites (100Km Away)':'# of wildfires (100Km Away)'}, inplace=True)
df3 = df3.merge(df, on=['BUILDING_ID___df2'], how='left')

In [64]:
#df3

In [65]:
len(df3['BUILDING_ID___df2'].unique())

17956

In [66]:
df3.columns.to_list()

['firename',
 'agency',
 'LATITUDE__df1',
 'LONGITUDE__df1',
 'BUILDING_ID___df1',
 'geometry',
 'index__df2',
 'CLIENT_ID',
 'CLIENT_NAME',
 'BUILDING_ID',
 'ADDRESS',
 'ADDRESS2',
 'CITY',
 'PROVINCE',
 'COUNTRY',
 'LATITUDE__df2',
 'LONGITUDE__df2',
 'POSTAL',
 'AREA_SQFT',
 'GROSS_SQFT',
 'RENTABLE_SQFT',
 'USABLE_SQFT',
 'OFFICE_SPACE_AREA',
 'LAND_AREA',
 'GEO_FENCE_UOM',
 'GEO_FENCE_RADIUS',
 'BUILDING_CREATED_DATE',
 'BUILDING_UPDATED_DATE',
 'PHYSICALLOCATION_PRIMARY_ADDRESS',
 'PHYSICALLOCATION_ADDRESS2',
 'PHYSICALLOCATION_CITY',
 'PHYSICALLOCATION_POSTALCODE',
 'HERITAGE_CODE',
 'PHYSICALLOCATION_CODE',
 'HERITAGE_DESIGNATION_DATE',
 'HERITAGE_DESIGNATION_DESCRIPTION',
 'DATUM',
 'PROPERTYUSAGE_DESC',
 'ACTIVEIND',
 'BUILDING_NAME',
 'Critical Environment Type',
 'BUILDING_ID___df2',
 '# of Sites (100Km Away)',
 'Distance (Km)',
 '# of wildfires (100Km Away)']

In [67]:
#wildfires

In [68]:
# Rename lat and long
wildfires.rename(columns={'LATITUDE':'LATITUDE__df2', 'LONGITUDE':'LONGITUDE__df2', 'agency':'PROVINCE'},inplace=True)

In [69]:
# rename province
df3.rename(columns={'PROVINCE__df2':'PROVINCE'},inplace=True)

In [70]:
df3['firename'].nunique()

1645

In [71]:
wildfires['BUILDING/WILDFIRES'] = 'WILDFIRES'

wildfires['BUILDING_ID_'] =  wildfires['firename']

df3['BUILDING_ID_'] = df3['BUILDING_ID']
df3['BUILDING/WILDFIRES'] = 'BUILDING'

In [72]:
df3 =pd.concat([df3,wildfires], axis=0)

In [73]:
df3['PROVINCE']

0        MB
1       NaN
2        MB
3        MB
4        MB
       ... 
1583     NV
1584     CA
1585     AZ
1586     AK
1587     AZ
Name: PROVINCE, Length: 115496, dtype: object

In [7]:
df3[df3['BUILDING_ID'].isnull()]

NameError: name 'df3' is not defined

In [75]:
# creating a column for building and wildfires
df3['BUILDING/WILDFIRES'] = np.where(df3['BUILDING_ID'].isnull(), 'WILDFIRES', df3['BUILDING/WILDFIRES'] )

In [76]:
# Adding wildfires provinces 
df3['PROVINCE'] = np.where(df3['PROVINCE'].isnull(), df3['agency'], df3['PROVINCE'] )

In [77]:
df3.isnull().sum()

firename                                 0
agency                                1667
LATITUDE__df1                         1667
LONGITUDE__df1                        1667
BUILDING_ID___df1                     1667
geometry                                 0
index__df2                            1667
CLIENT_ID                            33301
CLIENT_NAME                          33208
BUILDING_ID                          33208
ADDRESS                              33208
ADDRESS2                             33301
CITY                                 33208
PROVINCE                                 0
COUNTRY                              33208
LATITUDE__df2                            0
LONGITUDE__df2                           0
POSTAL                               33304
AREA_SQFT                            33301
GROSS_SQFT                           33301
RENTABLE_SQFT                        33293
USABLE_SQFT                          33301
OFFICE_SPACE_AREA                   115087
LAND_AREA  

In [78]:
df3['Distance (Km)'].describe()

count    113829.000000
mean         44.146842
std          21.024632
min           0.000000
25%          28.546569
50%          44.209976
75%          61.181133
max          89.836105
Name: Distance (Km), dtype: float64

In [79]:
# Creating distance Category

bins = [df3['Distance (Km)'].min(), 5 , 10, 25, 50]

names = ['Up to  5 Km', '5.1 - 10 Km','10.1 - 25 Km' ,'25.1 - 50 Km', '50.1 - 100 Km']

print(bins)

d = dict(enumerate(names, 1))

df3['DISTANCE_CATEGORY'] = np.vectorize(d.get)(np.digitize(df3['Distance (Km)'], bins))

[0.0, 5, 10, 25, 50]


In [80]:
df3['DISTANCE_CATEGORY'].unique()

array(['25.1 - 50 Km', 'Up to  5 Km', '10.1 - 25 Km', '50.1 - 100 Km',
       '5.1 - 10 Km'], dtype=object)

In [81]:
df3['Distance (Km)'][df3['DISTANCE_CATEGORY'].str.contains('Up to  5 Km')].describe()

count    3516.000000
mean        1.516603
std         1.732585
min         0.000000
25%         0.000000
50%         0.384796
75%         3.100390
max         4.998283
Name: Distance (Km), dtype: float64

In [82]:
df3['Distance (Km)'][df3['DISTANCE_CATEGORY'].str.contains('50.1 - 100 Km')].describe()

count    47333.000000
mean        64.797526
std          9.477275
min         50.001760
25%         56.917870
50%         63.934431
75%         71.302512
max         89.836105
Name: Distance (Km), dtype: float64

In [83]:
#df3.to_csv('Wildfires_nearest_neighbours_.csv', index=False)

In [8]:
df3

NameError: name 'df3' is not defined

## Saving file into a external directory

In [ ]:
df3.to_csv('****.csv', index=False)